# 1. Modern Pandas

## Get data

Just downloading data here. Feel free to ignore 😅.

In [1]:
import os
import zipfile

import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
headers = {
    "Referer": "https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time",
    "Origin": "https://www.transtats.bts.gov",
    "Content-Type": "application/x-www-form-urlencoded",
}

params = (
    ("Table_ID", "236"),
    ("Has_Group", "3"),
    ("Is_Zipped", "0"),
)

with open("modern-1-url.txt", encoding="utf-8") as f:
    data = f.read().strip()

os.makedirs("data", exist_ok=True)
dest = "data/flights.csv.zip"

if not os.path.exists(dest):
    r = requests.post(
        "https://www.transtats.bts.gov/DownLoad_Table.asp",
        headers=headers,
        params=params,
        data=data,
        stream=True,
    )

    with open("data/flights.csv.zip", "wb") as f:
        for chunk in r.iter_content(chunk_size=102400):
            if chunk:
                f.write(chunk)

In [3]:
zf = zipfile.ZipFile("data/flights.csv.zip")
fp = zf.extract(zf.filelist[0].filename, path="data/")
df = pd.read_csv(fp, parse_dates=["FL_DATE"]).rename(columns=str.lower)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450017 entries, 0 to 450016
Data columns (total 33 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   fl_date                450017 non-null  datetime64[ns]
 1   unique_carrier         450017 non-null  object        
 2   airline_id             450017 non-null  int64         
 3   tail_num               449378 non-null  object        
 4   fl_num                 450017 non-null  int64         
 5   origin_airport_id      450017 non-null  int64         
 6   origin_airport_seq_id  450017 non-null  int64         
 7   origin_city_market_id  450017 non-null  int64         
 8   origin                 450017 non-null  object        
 9   origin_city_name       450017 non-null  object        
 10  dest_airport_id        450017 non-null  int64         
 11  dest_airport_seq_id    450017 non-null  int64         
 12  dest_city_market_id    450017 non-null  int6

## Index

Two methods to get rows:

1. Use `.loc` for label-based indexing
2. Use `.iloc` for positional indexing

In [17]:
first = df.groupby("unique_carrier").first()

In [19]:
first.loc[["AA", "AS", "DL"], ["fl_date", "tail_num"]]

,fl_date,tail_num
unique_carrier,,
AA,2017-01-01,N3GNAA
AS,2017-01-01,N303AS
DL,2017-01-01,N942DL


In [20]:
first.iloc[[0, 1, 3], [0, 1]]

,fl_date,airline_id
unique_carrier,,
AA,2017-01-01,19805
AS,2017-01-01,19930
DL,2017-01-01,19790


## SettingWithCopy

Do not let the ends of two square brackets touch `][`. This does _not_ result in an an assignment to column `"b"`:

```python
# This is bad, do not do
f[f["a"] <= 3]["b"] = f[f["a"] <= 3]["b"] / 10
```

In [23]:
# Correct way
f = pd.DataFrame({"a": [1, 2, 3, 4, 5], "b": [10, 20, 30, 40, 50]})
f.loc[f["a"] <= 3, "b"] = f.loc[f["a"] <= 3, "b"] / 10
f

,a,b
0,1,1.0
1,2,2.0
2,3,3.0
3,4,40.0
4,5,50.0


## Multidimensional indexing

<div class="alert alert-block alert-info">
    <b>Techniques of note</b>
    <br><br>
    <li><b><code>pd.IndexSlice</code></b> for easy slicing of MultiIndexes</li>
</div>

In [24]:
hdf = df.set_index(
    ["unique_carrier", "origin", "dest", "tail_num", "fl_date"]
).sort_index()
hdf[hdf.columns[:4]].head()

airline_id  fl_num  \
unique_carrier origin dest tail_num fl_date                          
AA             ABQ    DFW  N3ABAA   2017-01-15       19805    2611   
                                    2017-01-29       19805    1282   
                           N3AEAA   2017-01-11       19805    2511   
                           N3AJAA   2017-01-24       19805    2511   
                           N3AVAA   2017-01-11       19805    1282   

                                                origin_airport_id  \
unique_carrier origin dest tail_num fl_date                         
AA             ABQ    DFW  N3ABAA   2017-01-15              10140   
                                    2017-01-29              10140   
                           N3AEAA   2017-01-11              10140   
                           N3AJAA   2017-01-24              10140   
                           N3AVAA   2017-01-11              10140   

                                                origin_airport_seq_id  
unique_carrier origin dest tail_num fl_date                            
AA             ABQ    DFW  N3ABAA   2017-01-15                1014003  
                                    2017-01-29                1014003  
                           N3AEAA   2017-01-11                1014003  
                           N3AJAA   2017-01-24                1014003  
                           N3AVAA   2017-01-11                1014003

Selecting outermost index

In [25]:
hdf.loc[["AA", "DL", "US"], ["dep_time", "dep_delay"]]

dep_time  dep_delay
unique_carrier origin dest tail_num fl_date                        
AA             ABQ    DFW  N3ABAA   2017-01-15     500.0        0.0
                                    2017-01-29     757.0       -3.0
                           N3AEAA   2017-01-11    1451.0       -9.0
                           N3AJAA   2017-01-24    1502.0        2.0
                           N3AVAA   2017-01-11     752.0       -8.0
...                                                  ...        ...
DL             XNA    ATL  N983AT   2017-01-11    1148.0      -11.0
                           N988DL   2017-01-26     556.0       -4.0
                           N989DL   2017-01-25     555.0       -5.0
                           N990DL   2017-01-15     604.0       -1.0
                           N995AT   2017-01-16    1152.0       -7.0

[142945 rows x 2 columns]

Selecting first two using a tuple `()`.

In [26]:
hdf.loc[(["AA", "DL", "US"], ["ORD", "DSM"]), ["dep_time", "dep_delay"]]

dep_time  dep_delay
unique_carrier origin dest tail_num fl_date                        
AA             DSM    DFW  N424AA   2017-01-23    1324.0       -3.0
                           N426AA   2017-01-25     541.0       -9.0
                           N437AA   2017-01-13     542.0       -8.0
                                    2017-01-23     544.0       -6.0
                           N438AA   2017-01-11     542.0       -8.0
...                                                  ...        ...
DL             ORD    SLC  N365NB   2017-01-18    1350.0       12.0
                           N368NB   2017-01-27    1351.0       13.0
                           N370NB   2017-01-20    1355.0       17.0
                           N374NW   2017-01-03    1846.0       -1.0
                           N987AT   2017-01-08    1914.0       29.0

[5582 rows x 2 columns]

Selecting only second index using `pd.IndexSlice`.

In [27]:
hdf.loc[pd.IndexSlice[:, ["ORD", "DSM"]], ["dep_time", "dep_delay"]]

dep_time  dep_delay
unique_carrier origin dest tail_num fl_date                        
AA             DSM    DFW  N424AA   2017-01-23    1324.0       -3.0
                           N426AA   2017-01-25     541.0       -9.0
                           N437AA   2017-01-13     542.0       -8.0
                                    2017-01-23     544.0       -6.0
                           N438AA   2017-01-11     542.0       -8.0
...                                                  ...        ...
WN             DSM    STL  N952WN   2017-01-29     854.0       -6.0
                           N954WN   2017-01-11    1736.0       -9.0
                           N956WN   2017-01-06    1736.0       -9.0
                           NaN      2017-01-16       NaN        NaN
                                    2017-01-17       NaN        NaN

[19466 rows x 2 columns]

In [28]:
pd.IndexSlice[:, ['ORD', 'DSM']]

(slice(None, None, None), ['ORD', 'DSM'])